# SI <font color='red'>YA ESTA GUARDADO</font> EL TERRENO CON SDF

## Leer el dataframe guardado

In [85]:
import numpy as np
dataframe = pd.read_csv('./files/terrain_sdf.csv')

In [86]:
dataframe

,x,y,z,density,distance
0,0,0,0,0.000000,7.348469
1,0,0,1,0.253968,6.708204
2,0,0,2,0.000000,6.164414
3,0,0,3,-0.253968,5.744563
4,0,0,4,0.000000,5.477226
...,...,...,...,...,...
7995,19,19,15,0.063492,5.000000
7996,19,19,16,-0.126984,5.099020
7997,19,19,17,0.000000,5.385165
7998,19,19,18,-0.063492,5.830952


### AHORA PODES PASAR  <font color='orange'>DIRECTO AL PASO 5 </font>

# SI <font color='red'>NO ESTA GUARDADO</font> EL TERRENO CON SDF EN UN ARCHIVO
Tenes que correr todo este archivo desde el paso 1

# 1. Generacion de cubo de tierra (datos)

In [1]:
!pip install noise

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [244]:
import numpy as np
import noise

def generate_terrain():
    shape = (20, 20, 20)
    scale = 2.0
    octaves = 6
    persistence = 0.5
    lacunarity = 2.0

    world = np.zeros(shape)

    for i in range(shape[0]):
        for j in range(shape[1]):
            for k in range(shape[2]):
                density = noise.pnoise3(i / scale,
                                        j / scale,
                                        k / scale,
                                        octaves=octaves,
                                        persistence=persistence,
                                        lacunarity=lacunarity,
                                        repeatx=1024,
                                        repeaty=1024,
                                        repeatz=1024,
                                        base=42)
                world[i][j][k] = density

    #plot_dataframe(world)
    return world

In [245]:
terrain = generate_terrain()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [246]:
terrain

array([[[ 0.        ,  0.25396827,  0.        , ...,  0.25396827,
          0.        ,  0.        ],
        [ 0.12698413,  0.19047619,  0.        , ...,  0.25396827,
          0.        , -0.06349207],
        [ 0.        ,  0.        ,  0.        , ...,  0.12698413,
          0.        , -0.25396827],
        ...,
        [-0.25396827, -0.19047619, -0.12698413, ...,  0.19047619,
          0.        , -0.38095239],
        [ 0.        ,  0.        ,  0.        , ...,  0.25396827,
          0.        , -0.25396827],
        [ 0.12698413,  0.25396827,  0.25396827, ...,  0.19047619,
          0.        ,  0.        ]],

       [[ 0.25396827,  0.06349207, -0.25396827, ...,  0.        ,
         -0.25396827,  0.        ],
        [ 0.25396827,  0.12698413, -0.19047619, ...,  0.06349207,
          0.        ,  0.06349207],
        [ 0.        ,  0.12698413,  0.        , ...,  0.12698413,
          0.25396827,  0.        ],
        ...,
        [-0.25396827, -0.12698413, -0.06349207, ...,  

# 2. Funcion para pasar de matriz a dataframe

In [269]:
import pandas as pd
def using_multiindex(A, columns):
    scale = 0.05
    shape = A.shape
    index = pd.MultiIndex.from_product([list(map(lambda number: number * scale, range(s))) for s in shape], names=columns)
    df = pd.DataFrame({'density': A.flatten()}, index=index).reset_index()
    return df

In [270]:
dataframe_terrain = using_multiindex(terrain, list('xyz'))

In [271]:
dataframe_terrain

,x,y,z,density
0,0.00,0.00,0.00,0.000000
1,0.00,0.00,0.05,0.253968
2,0.00,0.00,0.10,0.000000
3,0.00,0.00,0.15,-0.253968
4,0.00,0.00,0.20,0.000000
...,...,...,...,...
7995,0.95,0.95,0.75,0.063492
7996,0.95,0.95,0.80,-0.126984
7997,0.95,0.95,0.85,0.000000
7998,0.95,0.95,0.90,-0.063492


In [272]:
dataframe_terrain[['z']]

,z
0,0.00
1,0.05
2,0.10
3,0.15
4,0.20
...,...
7995,0.75
7996,0.80
7997,0.85
7998,0.90


# 3. Dataframe a SDF

In [251]:
from math import sqrt


def distanceSquared3D(point, other):
    return sqrt((point[0]-other[0])**2 + (point[1]-other[1])**2 + (point[2]-other[2])**2)


def getDistance(x, y, z, realInternalValue, internalValue = [0,1], columns = ['x','y','z','lineid']):
    coords = dataframe[columns]
    #print("internalValue: {} - realInternalValue: {}".format(internalValue, realInternalValue))
    
    realInternalValueIsAnInternalValue = internalValue[0] <= realInternalValue <= internalValue[1]
    #print(realInternalValueIsAnInternalValue)
    if realInternalValueIsAnInternalValue:
        # filtrar coords: quedarme con los distintos a internalValue
        coords = coords[~coords[columns[3]].between(internalValue[0], internalValue[1])]
    else:
        # filtrar coords: quedarme con los de igual a internalValue
        coords = coords[coords[columns[3]].between(internalValue[0], internalValue[1])]
    
    distances = coords.apply(lambda row: distanceSquared3D((x,y,z),(float(row[columns[0]]), float(row[columns[1]]), float(row[columns[2]]))), axis=1)
       
    min_distance = distances.min()
    if realInternalValueIsAnInternalValue:
        return -(min_distance)
    
    #print(min_distance)
    return min_distance
        
    
def dataframe_to_sdf(dataframe, internalValue = [0,1], columns = ['x','y','z','lineid']): #columns = [x, y, z, faultid, lineid]
    dataframe["distance"] = dataframe.apply(lambda x: getDistance(x[columns[0]], x[columns[1]], x[columns[2]], x[columns[3]], internalValue, columns), axis=1)
    return dataframe

In [273]:
#obtengo la densidad de uno de los puntos
dataframe_terrain.iloc[1].density

0.2539682686328888

In [274]:
import time

initial_time=time.time()
internalValue = [0.3,0.34] #dataframe_terrain.iloc[1].density
columns = ['x','y', 'z', 'density']
dataframe = dataframe_terrain
sdf = dataframe_to_sdf(dataframe_terrain, internalValue, columns)
end_time=time.time()
print("Calcularlo con 'map' tardo {} segundos".format(end_time-initial_time))
sdf

Calcularlo con 'map' tardo 29.93893313407898 segundos


,x,y,z,density,distance
0,0.00,0.00,0.00,0.000000,0.367423
1,0.00,0.00,0.05,0.253968,0.335410
2,0.00,0.00,0.10,0.000000,0.308221
3,0.00,0.00,0.15,-0.253968,0.287228
4,0.00,0.00,0.20,0.000000,0.273861
...,...,...,...,...,...
7995,0.95,0.95,0.75,0.063492,0.250000
7996,0.95,0.95,0.80,-0.126984,0.254951
7997,0.95,0.95,0.85,0.000000,0.269258
7998,0.95,0.95,0.90,-0.063492,0.291548


# 4. Guardar sdf en un archivo

In [275]:
sdf.to_csv (r'./files/terrain_sdf.csv', index = False, header=True)

# 5. Funcion para crear esferas

In [276]:
len(dataframe)

8000

In [277]:
dataframe

,x,y,z,density,distance
0,0.00,0.00,0.00,0.000000,0.367423
1,0.00,0.00,0.05,0.253968,0.335410
2,0.00,0.00,0.10,0.000000,0.308221
3,0.00,0.00,0.15,-0.253968,0.287228
4,0.00,0.00,0.20,0.000000,0.273861
...,...,...,...,...,...
7995,0.95,0.95,0.75,0.063492,0.250000
7996,0.95,0.95,0.80,-0.126984,0.254951
7997,0.95,0.95,0.85,0.000000,0.269258
7998,0.95,0.95,0.90,-0.063492,0.291548


In [278]:
dataframe.iloc[0]

x           0.000000
y           0.000000
z           0.000000
density     0.000000
distance    0.367423
Name: 0, dtype: float64

In [258]:
from math import sqrt

def distanceSquared3D(point, other):
    return sqrt((point[0]-other[0])**2 + (point[1]-other[1])**2 + (point[2]-other[2])**2)

def isInSphere(spheres, point):
    for sphere in spheres:
        if distanceSquared3D(point, sphere[0])< sphere[1]:
            return True
    return False

def makeSpheres(dataframe, max_spheres): #dataframe = (x,y,z,density,distance)
    coordinates_spheres = [] #[(Point,r)]
    for i in range(len(dataframe)):
        #print(i)
        distance = dataframe.iloc[i].distance
        if (distance > 0 and len(coordinates_spheres) < max_spheres):
            #print()
            coord = [dataframe.iloc[i].x, dataframe.iloc[i].y, dataframe.iloc[i].z]
            if(not isInSphere(coordinates_spheres, coord)):
                coordinates_spheres.append([coord, distance])

    return coordinates_spheres

def generateSpheres(dataframe, max_spheres, do_loop, use_brightness_level):
    spheresPoints = makeSpheres(dataframe, max_spheres)
    # plot(circles, use_brightness_level, do_loop)
    return spheresPoints

In [287]:
spheres = generateSpheres(dataframe, 20, do_loop=False, use_brightness_level=True)

In [288]:
spheres

[[[0.0, 0.0, 0.0], 0.3674234614174767],
 [[0.0, 0.0, 0.4], 0.18708286933869708],
 [[0.0, 0.0, 0.6000000000000001], 0.21213203435596423],
 [[0.0, 0.0, 0.8500000000000001], 0.18027756377319953],
 [[0.0, 0.15000000000000002, 0.75], 0.1224744871391589],
 [[0.0, 0.2, 0.35000000000000003], 0.15],
 [[0.0, 0.2, 0.9], 0.12247448713915889],
 [[0.0, 0.25, 0.5], 0.15],
 [[0.0, 0.25, 0.65], 0.158113883008419],
 [[0.0, 0.30000000000000004, 0.8], 0.07071067811865478],
 [[0.0, 0.35000000000000003, 0.15000000000000002], 0.21213203435596426],
 [[0.0, 0.35000000000000003, 0.9], 0.15000000000000002],
 [[0.0, 0.4, 0.4], 0.25495097567963926],
 [[0.0, 0.4, 0.7000000000000001], 0.18708286933869708],
 [[0.0, 0.5, 0.0], 0.18708286933869708],
 [[0.0, 0.5, 0.9500000000000001], 0.2958039891549808],
 [[0.0, 0.6000000000000001, 0.2], 0.12247448713915893],
 [[0.0, 0.6000000000000001, 0.6000000000000001], 0.07071067811865465],
 [[0.0, 0.65, 0.35000000000000003], 0.09999999999999998],
 [[0.0, 0.65, 0.5], 0.049999999999

# 6. Funcion para generar shader

In [222]:
def getValueOfIndex(array, index):
    return array[index]

def getX(array):
    return getValueOfIndex(array, 0)

def getY(array):
    return getValueOfIndex(array, 1)

def getZ(array):
    return getValueOfIndex(array, 2)

In [290]:
def generateShader(spheres):    
    f = "float sphereSD(vec3 p, vec3 pos, float r) { return length(pos - p) - r; }\nfloat sphereSDF(vec3 inputPoint) {\nreturn "
    sphere = spheres[0]
    prev = "sphereSD(inputPoint, vec3({0}, {1}, {2}), {3})".format(sphere[0][0]/x_max, sphere[0][1]/y_max, sphere[0][2]/z_max, float(sphere[1]/r_max))
    if (len(spheres) > 1):
        for i in range(1,len(spheres)):
            sphere = spheres[i]
            opt = "min(sphereSD(inputPoint, vec3({0}, {1}, {2}), {3}), {4})".format(sphere[0][0]/x_max, sphere[0][1]/y_max, sphere[0][2]/z_max, float(sphere[1]/r_max), prev)
            prev = opt
        f += opt
    else:
        f += prev
    f += ";\n}"
    print(f)

In [289]:
generateShader(spheres)

float sphereSD(vec3 p, vec3 pos, float r) { return length(pos - p) - r; }
float sphereSDF(vec3 inputPoint) {
return min(sphereSD(inputPoint, vec3(0.0, 0.65, 0.5), 0.04999999999999999), min(sphereSD(inputPoint, vec3(0.0, 0.65, 0.35000000000000003), 0.09999999999999998), min(sphereSD(inputPoint, vec3(0.0, 0.6000000000000001, 0.6000000000000001), 0.07071067811865465), min(sphereSD(inputPoint, vec3(0.0, 0.6000000000000001, 0.2), 0.12247448713915893), min(sphereSD(inputPoint, vec3(0.0, 0.5, 0.9500000000000001), 0.2958039891549808), min(sphereSD(inputPoint, vec3(0.0, 0.5, 0.0), 0.18708286933869708), min(sphereSD(inputPoint, vec3(0.0, 0.4, 0.7000000000000001), 0.18708286933869708), min(sphereSD(inputPoint, vec3(0.0, 0.4, 0.4), 0.25495097567963926), min(sphereSD(inputPoint, vec3(0.0, 0.35000000000000003, 0.9), 0.15000000000000002), min(sphereSD(inputPoint, vec3(0.0, 0.35000000000000003, 0.15000000000000002), 0.21213203435596426), min(sphereSD(inputPoint, vec3(0.0, 0.30000000000000004, 0.8), 0.

/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
